# LDA

LDA - это вероятностная модель для коллекций дискретных наборов данных, таких как текстовые корпуса. Также это тематическая модель, которая используется для обнаружения абстрактных тем из коллекции документов. Модель строит распределение Дирихле для каждой тематики с одним параметром, для каждого документа строит распределение Дирихле с другим параметром, и, в конечном счёте строит соответствия между словами из словаря корпуса и тематиками. Обзор LSI и LDA был ранее представлен в сравнительной характеристике методов векторизации:

https://docs.google.com/spreadsheets/d/1I4a-wPC-n3bFgrATCmp2D33UwkeczqTXHeirSiHPGFU/edit?usp=sharing

Рассмотрим применение модели LDA на примере.

Был самостоятельно собран датасет из произведений Есенина и Маяковского(по 30 стихотворений для каждого). Загрузим его и произведем частотную векторизацию(необходимо для модели LDA). 

In [1]:
import os
import pathlib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

In [2]:
def get_corpus(dirpath):
    path = pathlib.Path(dirpath)
    files = list(path.glob('*.txt'))
    corpus = []
    for name in files:
        with open(name) as file:
            corpus.append(file.read())
    return corpus

In [3]:
esenin_path = 'dataset/esenin/'
mayak_path = 'dataset/mayak/'
data1 = get_corpus(esenin_path)
data2 = get_corpus(mayak_path)
corpus = data1 + data2
len(corpus)

60

In [4]:
cv = CountVectorizer(max_features=2000)
X = cv.fit_transform(corpus)
X.shape

(60, 2000)

Теоретически, у нас две "тематики" - Есенин и Маяковский. Посмотрим, как тематики выделит LDA для данной выборки, и насколько результаты будет совпадать с точными.

In [5]:
lda = LatentDirichletAllocation(n_components=2, max_iter=100, random_state=0)
lda.fit(X)
transformed = lda.transform(X)

In [6]:
true_labels = np.array([0] * 30 + [1] * 30)

In [7]:
labels = list(map(lambda pair : np.argmax(pair), transformed))

In [8]:
precision = np.sum(np.where(np.equal(true_labels, labels), 1, 0)) / X.shape[0]
precision 

0.5333333333333333

Точность довольно низкая, но в данном случае это можно объяснить небольшим размером выборки. Попробуем ее улучшить с помощью вариации параметров.

In [9]:
cv = CountVectorizer(min_df=0.1, ngram_range=(1,3))
X = cv.fit_transform(corpus)
X.shape

(60, 149)

In [10]:
lda = LatentDirichletAllocation(n_components=2, max_iter=100, random_state=0)
lda.fit(X)
transformed = lda.transform(X)

In [11]:
labels = list(map(lambda pair : np.argmax(pair), transformed))

In [12]:
precision = np.sum(np.where(np.equal(true_labels, labels), 1, 0)) / X.shape[0]
precision 

0.6333333333333333

Таким образом, алгоритм LDA позволяет получить вероятности принадлежности документа к определенным абстрактным тематикам, что можно использовать для задачи обнаружения аномальных документов(например, в классах аномальных тематик и нормальных), а также для классификации текста. Помимо этого, полученные вероятности можно использовать как векторное представление текстов.